In [1]:
import os
import json
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
import re
import tensorflow as tf
from tensorflow import keras
from transformers import TextClassificationPipeline, AutoTokenizer, TFAutoModelForSequenceClassification

c:\Users\inigo\anaconda3\envs\HPE\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Enter your text

Config params:

In [2]:
# resources folder
resources_path = "../res"
datasets_path = os.path.join(resources_path, "datasets")
models_path = os.path.join(resources_path, "models")

In [3]:
num_classes = 24

model = TFAutoModelForSequenceClassification.from_pretrained(os.path.join(models_path, "bert_finetuned"))

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

pipe = TextClassificationPipeline(model=model, tokenizer=tokenizer, top_k = num_classes)

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

All the layers of TFBertForSequenceClassification were initialized from the model checkpoint at ../res\models\bert_finetuned.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForSequenceClassification for predictions without further training.


Enter text for first-aid recommendations:

In [4]:
input = "My kid has fallen overboard from my boat. We rescued from the water but now he is unconscious, what should I do?"

In [5]:
def predict_disease(text : str) -> str:
    return pipe(text)[0][:2]

In [6]:
conditions = predict_disease(input)
conditions

[{'label': 'drowning', 'score': 0.9311556220054626},
 {'label': 'fracture', 'score': 0.008220299147069454}]

In [7]:
conditions = conditions[0]['label']

Functions to search condition

In [8]:
def load_data(filename):
    # Load JSON data from a file
    with open(filename, 'r') as file:
        data = json.load(file)
    return data

def get_advice_by_tag(data, tag):
    # Search for the tag in the data and return the corresponding advice
    for entry in data['intents']:
        if entry['tag'].lower() == tag.lower():
            return entry['responses']
    return "No advice found for this tag."

In [9]:
filename = os.path.join(datasets_path, 'Medical_Aid.json')

data = load_data(filename)

advice = get_advice_by_tag(data, conditions)
print("Advice for", conditions, ":\n", advice[0])

Advice for drowning :
 1)Place your ear next to the person's mouth and nose. Do you feel air on your cheek? 2)Look to see if the person's chest is moving.If the Person is Not Breathing, Check Pulse. 3)Check the person's pulse for 10 seconds.If There is No Pulse, Start CPR.
